## Predicting daily changes in stock price overtime.

### Problem statement

Successful investing on stock exchange and generating profit out of it requires from a person a lot of commitment into collecting and analyzing the data concerning the investment targets. Additionally, the models used in price prediction can be robust and require deep knowledge in the field of statistics or finance. What is more, there are many factors, often connected with the investors expectations, that are hard to quantify and measure.

With all of that taken into consideration, investing your savings in stock exchange seems to be too timeconsuming and challenging for an individual. There exsists some alternatives e.g. investing in investment funds or hiring a financial advisor, but unfortunatelly they come with hefty costs in form of fees and provisions.

It would be an interesting challenge to build a machine learning model that would make accurate predictions about price changes of stock indices with little to no input from the user.

### Data


My only data source is Yahoo Finance website. The data itself is in a format of .csv file without any null values and consist of 6 features: the date, open/close and high/low prices and volume of stocks traded during that day. 

<img src="data.png">

The timespan for my data ranges from 14 to over 50 years, depending on the stock index. All of the indices used in testing the model are from NYSE or Nasdaq.

<img src="price_vs_volume.png">

### EDA

After taking a look at the data you can see at least a few issues:
 - there are only 5 features
 - all but one of the features are very highly correlated - see graph below
 - daily changes in the stock price are relatively small compared to the whole price so even models with high accuracy won't be able to give usefull results

<img src="feature_heatmap.png">

In order to tackle the first two of those issues I decided to create a few additional features in order to give my model more data to analyze:
 - Value of stocks traded each day obtained by multiplying the volume times average of high and low price
 - Daily change and percentage change of every feature to see how the values from consecutive days relate to each other
 - Data from previous days obtained by shifting the existing dataframe
 - Rolling means for every feature

By doing this I obtained a lot of new data that wasn't correlated with my starting features.

<img src="feature_heatmap_2.png">

Finally, in order to answer to the third problem with my data I decided to define my target as a percentage change in price in stead of the price itself. This way I have negated the effect of change in the average price of the index as well as made the distribution of my y closer to natural.

<img src="distribution.png">

### Model

During my test I used almost every basic machine learning model including: linear regression with lasso and ridge, logistic regression, decision tree, bagging, random forest, gradient boosting and neural net. Unfortunately none of those was able to give me usefull results.

Next, I decided to try Long Short-Term Memory Neural Network which is often used in case of a project containing a time series. LSTM model is able to "remember" values over arbitrary time intervals and because of that is able to grasp the relations between the data in a time series.

There are a few optimization questions that that arrise when using LSTM:
 - Like with every neural network, you need to decide how many layers to set up. Too shallow network might not be able to grasp the complexity of the model, too deep one on the other hand might overfit your training data and thus make your model inaccurate for the data it didn't see before
 - How much data to use for your training. If you don't feed your model enought data it might have a hard time finding the patterns neccesery for accurate predictions, too much data on the other hand will make the model "remember" too much irrelevant informations that doesn't have an impact on your target.
 - How big should be each batch of you data. In the LSTM model you divide your data into batches that are then presented to the model in turn. Different sizes of a batch and different number of batches in whole dataset will be make your model recognize various relations between the features and thus reflect on the predictions.

After performing extensive tests on Alphabet stocks price, I decided to build a model with 2 LSTM layers and 1 Dropuot layer in between. As for the dataset, I used a year worth of data (around 230 points) divided into 12 batches (11 for training set and 1 for test). It reflected the year/month division whitch might be the reason why this particular setting presented the best results.

<img src="roc.png">

### Tunning

After running my model many times I've noticed that usually the optimal threshold for deciding whether to buy a stock or go short on it was not exactly at 0 but somewhere in near neighbourhood. Because of that I decided to create a loop that would itterate through different thresholds, make the investment decision based on current threshold and calculate projected profit or loss on investment on my test set. Then I would save that information in order to show it on a graph.

<img src="threshold.png">

As you can see on the graph above, in this example the optimal threshold for prediction is around 0.5%.

Unfortunatelly because of high randomization of my predictions, I wasn't able to repeatedly produce comparable results with my model. In order to tackle that problem I decided to run my model 20 times for each prediction, save the results in a dataframe and base my final decision on how many of the 20 models recomended me to buy particular stock.

<img src="decisions.png">

Thanks to that I was able to increase my percentage of True Positives and True Negatives in my predictions and therefore increase performance of my model and projected profits.

<img src="roc_best.png">

### Simulations

In order to test my model I decided to run few simulations on different time periods: 2 weeks, 3 month and 1 year. All 3 simulations where run on Alphabet stock prices from year 2017 and 2018.

#### 2 weeks

While simulating 2 weeks of investment (10 business days) my model was able to predict 7 out of 10 correctly: 3 true positives and 4 true negatives. In case of remaining 3 days my model predicted increase of the price but in reality the price have declined - false positives

<img src="TF_table_2w.png">

<img src="roc_2w.png">

When calculating the actual profit on investment you also need to take into considerations transaction cost, which according to my research equals around \$5. Because of that, investing with my model requires substantial capital so that the transaction cost become marginal issue. In the case of 2 weeks simulation one would need to invest at least \$500 in order to break even and over \$2000 in order to earn the amount close to the projected income.

<img src="reco_profit_2w.png">

#### 3 months

While simulating 3 months of investment (62 business days) my model was able to predict 41 correctly (68%): 31 true positives and 10 true negatives. In case of remaining 21 days my model iccorectly predicted increase in price in 17 cases, and price drop in 4 cases.

<img src="TF_table_3m.png">

<img src="roc_3m.png">

Again, while calculating the actual profit after transaction cost it turns out that investing with my model requires at least \$400 in order to break even and over \$2000 in order to earn the amount close to the projected income.

<img src="reco_profit_3m.png">

#### 1 Year

While simulating a year of investment (251 business days) my model was able to predict 166 correctly (66%): 144 true positives and 22 true negatives. In case of remaining 85 days my model incorectly predicted increase in price in 79 cases and price drop in 6.

<img src="TF_table_2017.png">

<img src="roc_2017.png">

Again, while calculating the actual profit after transaction cost it turns out that investing with my model requires at least \$400 in order to break even and over \$3000 in order to earn the amount close to the projected income.

<img src="reco_profit_2017.png">

## Conclusion

Using LSTM Neural Network I was able to build a model that outperforms long term investment into stock with minimal input from the user (downloading financial data once a day). While that is true there are still ways to improve the model accuracy: in all 3 simulations the model was over optimistic: it predicted much more False Positives than False Negatives. Naturally the next step would be to look closser into those cases and try to determin what is reason for such an outcome.